# 目标窗口检测算法-NMS非极大值抑制 #

YOLO在最后的一个步骤就是对 $ S*S*(B*5+C) $ 个向量进行非极大值抑制（Non-max suppression），一开始不是太明白非极大值抑制是如何操作的，也就是不太清楚YOLO最后做完卷积后如何对求得向量进行预测，求得目标框框位置。

对YOLO代码分析完之后对其他步骤操作有了一个大致的认识之后，回顾最后一步非极大值抑制，发现非极大值抑制在R-CNN、Fast-RCNN都有用到的同样的概念，因此YOLO的论文并没有提到如何进行非极大值抑制。

其实在物体检测领域当中，非极大值抑制应用十分广泛，目的是为了消除多余的框，找到最佳的物体检测的位置。那么具体如何操作呢？如下图所示，有三个boundingbox，其中第一个绿色boundingbox的置信度是0.7，第二个绿色boundingbox的置信度是0.6，第三个绿色boundingbox的置信度是0.7。非极大值抑制就是在这三个框当中选出置信度最高，且最有可能代表是目标的boundingbox。

![](img/monroe0.jpg)

为了研究透NMS到底是如何操作的，我来随机定义了4个窗口，置信度分别是[0.5, 0.7, 0.6, 0.7]，对应上图的多了一个方框。算法的核心：

1. 把置信度最高的一个boundingbox(bbox)作为目标，然后对比剩下bbox与目标bbox之间的交叉区域
2. 如果交叉区域大于设定的阈值，那么在剩下的bbox中去除该bbox（即使该bbox的置信度与目标bbox的置信度一样）----这个操作就是抑制最大重叠区域
3. 把第二置信度高的bbox作为目标，重复1、2

In [100]:
import numpy as np

dets = np.array([
                [204, 102, 358, 250, 0.5],
                [257, 118, 380, 250, 0.7],
                [280, 135, 400, 250, 0.6],
                [255, 118, 360, 235, 0.7]])

thresh = 0.3

In [101]:
def nms(dets, thresh):
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]
    
    areas = (x2 - x1 + 1) * (y2 - y1 + 1) # 每个boundingbox的面积
    order = scores.argsort()[::-1] # boundingbox的置信度排序

    keep = [] # 用来保存最后留下来的boundingbox
    while order.size > 0:     
        i = order[0] # 置信度最高的boundingbox的index
        keep.append(i) # 添加本次置信度最高的boundingbox的index
        
        # 当前bbox和剩下bbox之间的交叉区域
        # 选择大于x1,y1和小于x2,y2的区域
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        # 当前bbox和其他剩下bbox之间交叉区域的面积
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        
        # 交叉区域面积 / (bbox + 某区域面积 - 交叉区域面积)
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        #保留交集小于一定阈值的boundingbox
        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]
        
    return keep

print nms(dets, thresh)

[3]


最后的效果就是得到开始定义的4个bbox中的第4个(3):

![](img/monroe1.jpg)